# Collectl CPU Log Analysis

## Functionalities
- Plot CPU utilization graphs.

## Input
Log files are read from a directory in `../data`. This directory is assumed to have the following structure:
```
logs/
  [node-1]/
    collectl.tar.gz
  ...
  [node-n]/
    collectl.tar.gz
```
A tarball `collectl.tar.gz` contains log files. The log file extension identifies the type of resource monitored:
- `.cpu.gz`: CPU monitoring log file.
- `.numa.gz`: memory monitoring log file.
- `.dsk.gz`: disk I/O monitoring log file.

## Notebook Configuration

In [ ]:
########## GENERAL
# Name of the directory in `../data`
EXPERIMENT_DIRNAME = "BuzzBlogBenchmark_2022-01-15-18-52-05"
# Ramp up duration (in sec)
RAMP_UP_DURATION = 720
# Ramp down duration (in sec)
RAMP_DOWN_DURATION = 360

########## CPU
# Analyzed metric (options: "user", "nice", "system", "wait", "irq", "soft",
# "steal", "idle", "total", "guest", "guest_n", "intrpt")
COLLECTL_CPU_METRIC = "total"
# Filter CPU cores
COLLECTL_CPU_CORES = None

## Notebook Setup

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import sys
import warnings
warnings.filterwarnings("ignore")

sys.path.append(os.path.abspath(os.path.join("..")))
from parsers.collectl_parser import CollectlParser
from utils.utils import *

## Log Parsing

In [ ]:
# Get experiment time range, excluding ramping periods
experiment_start_time = get_experiment_start_time(EXPERIMENT_DIRNAME)
start_time = experiment_start_time + np.timedelta64(RAMP_UP_DURATION, 's')
experiment_end_time = get_experiment_end_time(EXPERIMENT_DIRNAME)
end_time = experiment_end_time - np.timedelta64(RAMP_DOWN_DURATION, 's')

In [ ]:
# Build data frames
cpu = pd.concat([
    pd.DataFrame.from_dict(CollectlParser(logfile, "cpu", start_time, end_time).parse()).assign(node_name=node_name)
    for node_name, logfile in get_collectl_cpu_logfiles(EXPERIMENT_DIRNAME)
], ignore_index=True)

In [ ]:
# Filter data frames
cpu = cpu[(cpu["hw_metric"] == COLLECTL_CPU_METRIC)]
if COLLECTL_CPU_CORES:
    cpu = cpu[(cpu["hw_no"].isin(COLLECTL_CPU_CORES))]

In [ ]:
cpu["hw_no"].unique()

In [ ]:
# (Re) Build columns
cpu["timestamp"] = cpu.apply(lambda r: (r["timestamp"] - start_time).total_seconds(), axis=1)
cpu["window"] = cpu.apply(lambda r: int(r["timestamp"]), axis=1)
cpu["value"] = cpu.apply(lambda r: float(r["value"]), axis=1)

In [ ]:
# (Re) Create index
cpu.set_index("timestamp", inplace=True)
cpu.sort_index(inplace=True)

In [ ]:
# Get values
node_names = get_node_names(EXPERIMENT_DIRNAME)

## CPU Monitoring

In [ ]:
# Plot CPU utilization (1-sec granularity)
fig = plt.figure(figsize=(24, len(node_names) * 12))
for (i, node_name) in enumerate(node_names):
    df = cpu[(cpu["node_name"] == node_name)]
    df = df.groupby(["window"])["value"].mean()
    ax = fig.add_subplot(len(node_names), 1, i + 1)
    ax.set_xlim((df.index.min(), df.index.max()))
    ax.set_ylim((0, 100))
    ax.grid(alpha=0.75)
    df.plot(ax=ax, kind="line", title="%s - CPU Utilization" % node_name, xlabel="Time (seconds)",
            ylabel="%s (%%)" % COLLECTL_CPU_METRIC, grid=True, legend=False, yticks=range(0, 101, 10))

In [ ]:
########## LOCAL CONFIG
# Minimum time (in seconds)
MIN_TIME = None
# Maximum time (in seconds)
MAX_TIME = None

# Plot CPU utilization (millisec granularity)
if MIN_TIME and MAX_TIME:
    fig = plt.figure(figsize=(24, len(node_names) * 12))
    for (i, node_name) in enumerate(node_names):
        df = cpu[(cpu["node_name"] == node_name)]
        df = df[(df.index >= MIN_TIME) & (df.index <= MAX_TIME)]
        df = df.groupby(["timestamp", "hw_no"])["value"].mean()
        df = df.unstack()
        ax = fig.add_subplot(len(node_names), 1, i + 1)
        ax.set_xlim((df.index.min(), df.index.max()))
        ax.set_ylim((0, 100))
        ax.grid(alpha=0.75)
        df.plot(ax=ax, kind="line", title="%s - CPU Utilization" % node_name, xlabel="Time (seconds)",
                ylabel="%s (%%)" % COLLECTL_CPU_METRIC, grid=True, legend=True, yticks=range(0, 101, 10))